In [1]:
import operator
import random
import requests
import time
import csv
import os
import pandas as pd
import seaborn as sns
import numpy as np
import networkx as nx
from scipy import stats
from ast import literal_eval
from matplotlib import pyplot as plt
from statsmodels.stats import diagnostic
from datetime import datetime
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.model_selection import train_test_split

# Data standard scaler
from sklearn.preprocessing import StandardScaler

# Random train-test split
from sklearn.model_selection import train_test_split

# Metrics and model selection
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, auc, roc_curve, precision_recall_curve
from sklearn.model_selection import cross_val_predict

# Classifiers
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier

from scipy.stats import ttest_ind

#To visualize on of the RandomForest decision trees
#from sklearn import tree

#colors used when comparing genders
PURPLE = 'purple'
ORANGE = '#ffb82c'

#colors used when generic comparisons
BLUE = '#0173b2'
RED = '#d55e00'

#print(sns.color_palette("colorblind").as_hex())
#sns.color_palette("colorblind")

In [2]:
from IPython.core.display import HTML
HTML("""
<style>
.output_png {
    display: table-cell;
    text-align: center;
    vertical-align: middle;
}
</style>
""")

# 1. Load the data
<a class="anchor" id="load"></a>

In [3]:
DATA_PATH = './data'
MOVIE_PATH = os.path.join(DATA_PATH, 'american_movies_cleaned.tsv')
CHAR_PATH = os.path.join(DATA_PATH, 'characters_cleaned.tsv')
THE_NUMBERS_PATH = os.path.join(DATA_PATH, 'the_numbers_lifetime_cumulative.tsv')
THE_NUMBERS_PATH_YEARLY = os.path.join(DATA_PATH, 'the_numbers_cleaned_yearly.tsv')

## 1.1 The numbers cumulated

In [16]:
df_numbers_cum=pd.read_csv(THE_NUMBERS_PATH, sep='\t').set_index('actor_name')

#drop all 0 rows
df_numbers_cum = df_numbers_cum.loc[~(df_numbers_cum==0).all(axis=1)]
df_numbers_cum.drop(['1978', '1979'], axis=1, inplace=True)
df_numbers_cum.head(2)

,1980,1981,1982,1983,1984,1985,1986,1987,1988,1989,...,2015,2016,2017,2018,2019,2020,2021,2022,min_year,min_year_score
actor_name,,,,,,,,,,,,,,,,,,,,,
Jane Fonda,440.0,539.0,539.0,539.0,539.0,539.0,602.0,602.0,602.0,602.0,...,696.0,696.0,696.0,755.0,755.0,755.0,755.0,755.0,1980,440.0
Clint Eastwood,360.0,360.0,446.0,540.0,707.0,789.0,872.0,872.0,944.0,972.0,...,1672.0,1672.0,1672.0,1743.0,1743.0,1743.0,1779.0,1779.0,1980,360.0


## 1.2 The numbers data yearly

In [23]:
df_numbers_yearly = pd.read_csv(THE_NUMBERS_PATH_YEARLY, sep='\t').set_index('actor_name')
df_numbers_yearly.columns = df_numbers_yearly.columns.astype(int)
df_numbers_yearly.head(2)

,1980,1981,1982,1983,1984,1985,1986,1987,1988,1989,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
actor_name,,,,,,,,,,,,,,,,,,,,,
Jane Fonda,99.0,99.0,0.0,0.0,0.0,0.0,63.0,0.0,0.0,0.0,...,0.0,16.0,0.0,0.0,0.0,59.0,0.0,0.0,0.0,0.0
Clint Eastwood,174.0,0.0,86.0,94.0,167.0,82.0,83.0,0.0,72.0,28.0,...,0.0,0.0,0.0,0.0,0.0,71.0,0.0,0.0,36.0,0.0


In [5]:
columns = df_numbers_yearly.columns
def get_min_year(row):
    for x in columns[:-2]:
        if row[x]>0:
            return x
    return 0

In [6]:
def get_min_year_score(row):
    for x in columns[:-2]:
        if row[x] >0:
            return float(row[x])
    return 0

In [7]:
df_numbers_yearly['min_year'] = df_numbers_yearly.apply(lambda row: get_min_year(row), axis=1)
df_numbers_yearly['min_year_score'] = df_numbers_yearly.apply(lambda row: get_min_year_score(row), axis=1)
df_numbers_yearly.head(3)

,1980,1981,1982,1983,1984,1985,1986,1987,1988,1989,...,2015,2016,2017,2018,2019,2020,2021,2022,min_year,min_year_score
actor_name,,,,,,,,,,,,,,,,,,,,,
Jane Fonda,99.0,99.0,0.0,0.0,0.0,0.0,63.0,0.0,0.0,0.0,...,0.0,0.0,0.0,59.0,0.0,0.0,0.0,0.0,1980,99.0
Clint Eastwood,174.0,0.0,86.0,94.0,167.0,82.0,83.0,0.0,72.0,28.0,...,0.0,0.0,0.0,71.0,0.0,0.0,36.0,0.0,1980,174.0
John Belushi,92.0,125.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1980,92.0


In [8]:
df_numbers_yearly.reset_index(inplace=True)

for y in range(1980, 2023):
    range_col = list(np.arange(1980, y+1))
    columns = range_col + ['actor_name']
    df_tmp = df_numbers_yearly[columns].set_index('actor_name')
    series_tmp = df_tmp.apply(lambda row: np.count_nonzero(row[range_col]), axis=1).reset_index()
    series_tmp.rename({0: 'nb_ranked_'+str(y)}, axis=1, inplace=True)
    df_numbers_yearly = df_numbers_yearly.merge(series_tmp, on='actor_name', how='left')

In [11]:
df_numbers_yearly.set_index('actor_name', inplace=True)
df_numbers_yearly.head()

,1980,1981,1982,1983,1984,1985,1986,1987,1988,1989,...,nb_ranked_2013,nb_ranked_2014,nb_ranked_2015,nb_ranked_2016,nb_ranked_2017,nb_ranked_2018,nb_ranked_2019,nb_ranked_2020,nb_ranked_2021,nb_ranked_2022
actor_name,,,,,,,,,,,,,,,,,,,,,
Jane Fonda,99.0,99.0,0.0,0.0,0.0,0.0,63.0,0.0,0.0,0.0,...,4,5,5,5,5,6,6,6,6,6
Clint Eastwood,174.0,0.0,86.0,94.0,167.0,82.0,83.0,0.0,72.0,28.0,...,19,19,19,19,19,20,20,20,21,21
John Belushi,92.0,125.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2,2,2,2,2,2,2,2,2,2
Burt Reynolds,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2,2,2,2,2,2,2,2,2,2
Goldie Hawn,170.0,0.0,81.0,0.0,66.0,0.0,66.0,59.0,0.0,0.0,...,11,11,11,11,12,12,12,12,12,12


In [21]:
df_numbers_yearly[1980] = df_numbers_cum['1980']

In [22]:
OUTPUT_FILE_NUMBERS = 'the_numbers_yearly_score.tsv'
OUTPUT_PATH = os.path.join(DATA_PATH, OUTPUT_FILE_NUMBERS)
df_numbers_yearly.to_csv(OUTPUT_PATH, sep="\t")